# <Step 1> 크롤링 : 웹 데이터 가져오기

### [BeautifulSoup을 이용해 웹 크롤링 하기]

In [4]:
# pip install selenium beautifulsoup4 로 라이브러리 설치
# pip install webdriver_manager로 라이브러리 설치
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import re
# pip uninstall konlpy==0.5.1 jpype1==0.6.3 Jpype1-py3
# pip install konlpy==0.5.1 jpype1==0.6.3 Jpype1-py3
#pip install konlpy==0.5.1
# pip install jpype==0.6.3
# pip install Jpype1-py3
from konlpy.tag import Okt
from collections import Counter


# 크롤링할 사이트 주소를 정의합니다.
source_url = "https://namu.wiki/RecentChanges"

#options = webdriver.ChromeOptions()
#options.add_experimental_option("excludeSwitches", ["enable-logging"])
# 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
#driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)  # for Windows
driver = webdriver.Chrome(ChromeDriverManager().install())  # for Windows
driver.get(source_url)
driver.implicitly_wait(10)
#table_rows = driver.find_elements(By.XPATH,'//*[@id="C6Rc9QlVe"]/div[2]/div/div/div/div/div/article/div[3]/div/div/div/div[1]/div/div/table/tbody/tr/td/a')
table_rows = driver.find_elements(By.XPATH,'//*[@id="C6Rc9QlVe"]/div[2]/div/div/div/div/div/article/div[3]/div/div/div/div[1]/div/div/table/tbody/tr')
print(table_rows)
page_urls = []

#for i in range(0,len(table_rows)): # 시간이 너무 오래 걸림
for i in range(0,5):    # 어차피 맨 밑에 상위 다섯번째까지만 출력하면 되니 5번만 반복.
    first_td=table_rows[i].find_elements(By.TAG_NAME,"td")
    td_url=first_td[0].find_elements(By.TAG_NAME,"a")
    if len(td_url) >0 :
        page_url=td_url[0].get_attribute("href")
        #page_url=table_rows[0].get_attribute("href")   # a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성합니다.
        print(page_url)
        if 'png' not in page_urls:
            page_urls.append(page_url)

# 중복 url을 제거합니다.
page_urls = list(set(page_urls))
print(page_urls)
#for page in page_urls[:3]:
#    print(page)





C:\Users\GR\AppData\Local\Temp\ipykernel_11912\421460542.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())  # for Windows


[<selenium.webdriver.remote.webelement.WebElement (session="0d163cde8f34a577badd01c5f20dbb09", element="e3b3e27c-1a4b-42a5-984a-37798d8bab3a")>, <selenium.webdriver.remote.webelement.WebElement (session="0d163cde8f34a577badd01c5f20dbb09", element="7a7ff7d1-5641-4999-84ce-41c44f8cb36f")>, <selenium.webdriver.remote.webelement.WebElement (session="0d163cde8f34a577badd01c5f20dbb09", element="cffc4165-498c-4023-8e0d-8fde8aa1f7e6")>, <selenium.webdriver.remote.webelement.WebElement (session="0d163cde8f34a577badd01c5f20dbb09", element="b4b41cce-1878-4f1a-816a-c65979928cea")>, <selenium.webdriver.remote.webelement.WebElement (session="0d163cde8f34a577badd01c5f20dbb09", element="54b3fa9d-15b0-479f-8d3d-e3fa10166e9a")>, <selenium.webdriver.remote.webelement.WebElement (session="0d163cde8f34a577badd01c5f20dbb09", element="300e1809-114a-4c45-8e19-507504cca843")>, <selenium.webdriver.remote.webelement.WebElement (session="0d163cde8f34a577badd01c5f20dbb09", element="a2ebd101-07f1-47a7-9aa6-d382c1ff

### [나무위키의 최근 변경 데이터 크롤링하기]

In [5]:

columns = ['title','category','content_text']
df=pd.DataFrame(columns=columns)

for page_url in page_urls:
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(10)
    driver.get(page_url)
    req=driver.page_source
    soup=BeautifulSoup(req,"html.parser")
    contents_table=soup.find(attrs={"class":"SVuqC-pU"})
    title=contents_table.find_all("h1")[0]

    if len(contents_table.find_all("ul"))>0:
        category=contents_table.find_all("ul")[0]
    else:
        category=None

    content_paragraphs=contents_table.find_all(name="div", attrs={"class":"UtCm7-qJ"})
    content_corpus_list=[]

    if title is not None:
        row_title=title.text.replace("\n"," ")
    else:
        row_title=""
    
    if content_paragraphs is not None:
        for paragraghs in content_paragraphs:
            if paragraghs is not None:
                content_corpus_list.append(paragraghs.text.replace("\n"," "))
            else:
                content_corpus_list.append("")
    else:
        content_corpus_list("")

    if category is not None:
        row_category=category.text.replace("\n"," ")
    else:
        row_category=""

    row=[row_title,row_category,"".join(content_corpus_list)]
    series=pd.Series(row, index=df.columns)
    df=df.append(series, ignore_index=True)




    driver.close()
df.head(5)

C:\Users\GR\AppData\Local\Temp\ipykernel_11912\2721062881.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\GR\AppData\Local\Temp\ipykernel_11912\2721062881.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(series, ignore_index=True)


,title,category,content_text
0,파일:United_Nations_Interim_Administration_Missi...,파일/로고,출처https://en.wikipedia.org/wiki/File:United_Na...
1,제네시스 G70,제네시스/생산차량2017년 출시중형차스포츠 세단디지털 키/탑재차량,2017년 9월 15일에 공개된 제네시스의 중형 스포츠 세단.근본부터 독창적인. 우...
2,파일:UNMIK 로고.svg,파일/로고,출처출처를 반드시 삽입해 주세요.날짜이미지가 만들어진 날짜를 삽입해 주세요.저작자이...


### [텍스트 데이터 전처리 하기]

In [8]:
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글의 정규표현식을 나타냅니다.
    result = hangul.sub('', text)
    return result

print(text_cleaning(df["content_text"][0]))

출처날짜이미지가 만들어진 날짜를 삽입해 주세요저작자이미지의 저작자를 삽입해 주세요저작권이미지의 저작권과 관련된 기타 정보를 삽입해 주세요기타기타 정보가 있으면 삽입해 주세요이미지의 자세한 설명을 적어 주세요


### [모든 데이터 전처리 적용하기]

In [9]:
df["title"]=df["title"].apply(lambda x:text_cleaning(x))
df["category"]=df["category"].apply(lambda x:text_cleaning(x))
df["content_text"]=df["content_text"].apply(lambda x:text_cleaning(x))

print(df.head(5))


    title                        category  \
0     파일                             파일로고   
1  제네시스    제네시스생산차량년 출시중형차스포츠 세단디지털 키탑재차량   
2  파일 로고                             파일로고   

                                        content_text  
0  출처날짜이미지가 만들어진 날짜를 삽입해 주세요저작자이미지의 저작자를 삽입해 주세요저...  
1  년 월 일에 공개된 제네시스의 중형 스포츠 세단근본부터 독창적인 우리만의 방식으로주...  
2  출처출처를 반드시 삽입해 주세요날짜이미지가 만들어진 날짜를 삽입해 주세요저작자이미지...  


### [말뭉치 만들기] 

In [10]:
title_corpus="".join(df["title"].tolist())
category_corpus="".join(df["category"].tolist())
content_corpus="".join(df["content_text"].tolist())

print(title_corpus)
print(category_corpus)
print(content_corpus)

파일 제네시스  파일 로고 
파일로고제네시스생산차량년 출시중형차스포츠 세단디지털 키탑재차량파일로고
출처날짜이미지가 만들어진 날짜를 삽입해 주세요저작자이미지의 저작자를 삽입해 주세요저작권이미지의 저작권과 관련된 기타 정보를 삽입해 주세요기타기타 정보가 있으면 삽입해 주세요이미지의 자세한 설명을 적어 주세요년 월 일에 공개된 제네시스의 중형 스포츠 세단근본부터 독창적인 우리만의 방식으로주요 엔진 라인업으로는  싱글터보 가솔린  트윈터보 가솔린  디젤이며 파워트레인은 후륜용 단 자동변속기가 들어갔다루머와 달리 제네시스 뉴욕 컨셉은 도 반영되지 않을 예정이다 현대차 내부에서는 의 양산단계 직전까지 뉴욕컨셉의 존재조차 몰랐다고 한다 뉴욕컨셉이 발표되었을 때는 이미 의 최종 디자인 프로세스까지 다 끝난 단계였으며 제네시스 뉴욕 컨셉은 미국 스튜디오의 단독 컨셉 모델이다 때문에 시간상의 문제도 그렇고  양산차는 뉴욕컨셉과 전혀 관계없이 진행됐다 이미 공개된 실루엣만 봐도 알듯이 형상 자체가 다르다 오히려   컨셉의 앞모습과 매우 비슷하다스팅어의 플랫폼을 공유하는 은 스팅어보다 비싸지만 차체 크기는 작다 또 가격대가 스팅어보다 높은 만큼 알루미늄 소재의 사용 비율을 높인다는 루머가 있었으나 스팅어와 큰 차이가 없는 것으로 드러났다년 월 일부터 일까지 선착순으로 프라이빗 쇼룸 이벤트를 신청하여 관람이 가능했는데 서울 부산 대구 인천 등 전국에서 진행되었으며 서울의 경우 강남구 언주로에 위치한 현대모터스튜디오에서 관람이 가능했다년 기준으로 기본 트림 가격대는 만 원으로 스팅어보다 트림당 약 만원에서 만원 가량 더 비싸다 또한 배기량 과  트림의 구성품과 선택 가능한 옵션에 차이를 뒀던 스팅어와 달리 은 옵션 선택폭이 넓으며 엔진 별 구성품 차별도 적다다만 스팅어에 있는 전동식 파워 트렁크 시스템 컷 스티어링 휠이 배제되고 전자 제어 서스펜션이 상급 스포츠  터보 트림에만 채택된 것은 단점으로 꼽힌다초기에 차동 기어 장치와 전자식 서스펜션이 전 트림에 들어간다는 루머가 있어 기대

### [konlpy를 이용해 키워드 추출하기]

In [11]:
nouns_tagger = Okt()
nouns = nouns_tagger.nouns(content_corpus)
count = Counter(nouns)
print(count)
remove_char_counter=Counter({x : count[x] for x in count if len(x)>1})
print(remove_char_counter)

Counter({'제네시스': 57, '것': 51, '브랜드': 46, '스팅어': 44, '모델': 42, '이': 42, '차량': 42, '수': 36, '년': 32, '등': 30, '더': 30, '차': 30, '월': 29, '의': 27, '스포츠': 26, '경우': 26, '브레이크': 24, '자동차': 23, '옵션': 22, '트림': 21, '디자인': 20, '가격': 20, '현대': 20, '패드': 20, '문제': 19, '적용': 19, '성능': 19, '좌석': 19, '중': 17, '프리미엄': 16, '비': 16, '출시': 16, '터보': 15, '만원': 15, '가치': 15, '디스크': 15, '이미지': 14, '일': 14, '엔진': 14, '파워': 14, '시리즈': 14, '등장': 14, '고성능': 14, '앞': 13, '만': 13, '추가': 13, '기존': 13, '디젤': 12, '때문': 12, '사용': 12, '고급': 12, '변경': 12, '정도': 12, '가성': 12, '를': 12, '시장': 12, '렉서스': 12, '페이스': 12, '북미': 12, '부분': 11, '실내': 11, '판매': 11, '순정': 11, '개발': 11, '카': 11, '리프트': 11, '로터': 11, '방식': 10, '달리': 10, '현대차': 10, '때': 10, '은': 10, '차이': 10, '또한': 10, '서스펜션': 10, '윈도우': 10, '기능': 10, '사실': 10, '공간': 10, '세단': 10, '세대': 10, '삽입': 9, '공개': 9, '중형': 9, '플랫폼': 9, '크기': 9, '다년': 9, '기본': 9, '이후': 9, '급': 9, '인': 9, '브렘보': 9, '리퍼': 9, '수준': 9, '실제': 9, '볼': 9, '그': 9, '판매량': 9, '쿠페': 9, '실적': 9, '사양': 9